In [1]:
import os
from collections import defaultdict
import sys
import time
import json
import psutil
import argparse
import operator
import pandas as pd
sys.path.append("/nykaa/api")
from pas.v2.utils import Utils

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
orders_query = "SELECT order_id, product_id, mrp FROM sales_flat_order_item WHERE order_id <> 0 AND mrp > 1 and product_type='simple' and created_at >= '2018-04-01 00:00:00'; "

In [3]:
rows = DiscUtils.fetchResultsInBatch(DiscUtils.nykaaMysqlConnection(), orders_query, 10000)

df = pd.DataFrame()
df = df.append(rows)
len(rows), len(df)

(8541489, 8541489)

In [4]:
df.columns = ['order_id', 'product_id', 'mrp']

In [5]:
df.head()

,order_id,product_id,mrp
0,2238584,71314,175.0000
1,2238584,71312,175.0000
2,2238584,28033,149.0000
3,2238584,7877,299.0000
4,7826990,31081,695.0000


In [6]:
query = "select child_id, parent_id from catalog_product_relation"

rows = DiscUtils.fetchResultsInBatch(DiscUtils.nykaaMysqlConnection(), query, 50000)
child_2_parent = {row[0]: row[1] for row in rows}

In [7]:
with open("child_product_2_parent.json", "r+") as f:
    child_2_parent.update({int(key): int(value) for key, value in json.load(f).items()})

In [8]:
from multiprocessing import Pool
import numpy as np

def parallelize_dataframe(df, func, num_partitions, num_cores):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [20]:
def convert_2_parent_product(data):
    data['product_id'] = data.apply(lambda row: child_2_parent.get(row['product_id'], row['product_id']), axis=1)
    return data

df_parented = parallelize_dataframe(df, convert_2_parent_product, 40, 16)

In [21]:
df_parented.head()

,order_id,product_id,mrp
0,2238584,71309,175.0000
1,2238584,71309,175.0000
2,2238584,28012,149.0000
3,2238584,7878,299.0000
4,7826990,31081,695.0000


In [22]:
df_parented = df_parented.filter(['product_id', 'mrp'])

In [23]:
df_grouped = df_parented.groupby(['product_id']).agg({'mrp': 'sum'}).reset_index()

In [24]:
df_grouped.head()

,product_id,mrp
0,250,20180.0000
1,254,5164900.0000
2,261,1446300.0000
3,310,541525.0000
4,317,407015.0000


In [25]:
df_grouped.head()

,product_id,mrp
0,250,20180.0000
1,254,5164900.0000
2,261,1446300.0000
3,310,541525.0000
4,317,407015.0000


In [27]:
df_grouped_sorted = df_grouped.sort_values(by=['mrp'], ascending=False)

In [29]:
df_top_200 = df_grouped_sorted[:200]

In [30]:
df_top_200.head()

,product_id,mrp
23880,208047,26449500.0000
37266,303851,25197700.0000
31377,265965,24127930.0000
6745,31074,23832700.0000
29140,248368,21641174.0000


In [31]:
df_top_200 = df_top_200.filter(['product_id'])

In [36]:
product_ids = []
for row in df_top_200.to_dict(orient='records'):
    product_ids.append(int(row['product_id']))

In [37]:
product_ids

[208047,
 303851,
 265965,
 31074,
 248368,
 71406,
 8997,
 40862,
 5997,
 152930,
 809,
 73137,
 2374,
 61421,
 208127,
 111777,
 90470,
 37835,
 8085,
 34547,
 189967,
 18430,
 28012,
 27269,
 208079,
 116489,
 1287,
 14960,
 88999,
 31069,
 90344,
 13369,
 73011,
 89231,
 28760,
 114009,
 294926,
 110227,
 262168,
 7577,
 129934,
 81399,
 44942,
 108481,
 273051,
 13386,
 171236,
 48066,
 28273,
 2145,
 304242,
 21795,
 44879,
 85190,
 23215,
 255368,
 288560,
 37894,
 220214,
 300065,
 216554,
 35523,
 257861,
 79623,
 160488,
 31362,
 806,
 18423,
 60555,
 119776,
 28359,
 13366,
 116381,
 209006,
 18020,
 220217,
 254,
 209005,
 25717,
 90475,
 276637,
 36456,
 7925,
 3056,
 110224,
 7743,
 155416,
 7919,
 1313,
 13378,
 218602,
 110406,
 28720,
 222549,
 331327,
 229917,
 7878,
 183012,
 28299,
 92040,
 5989,
 63379,
 2581,
 20759,
 21888,
 6812,
 34039,
 18412,
 282679,
 173611,
 110238,
 299544,
 11501,
 262769,
 183962,
 11502,
 10976,
 68377,
 196207,
 112646,
 5607,
 6083,


In [38]:
with open('top_200_products_by_revenue.json', 'w') as file:
    json.dump(product_ids, file)

In [39]:
df_grouped[:1]

,product_id,mrp
0,250,20180.0000


In [43]:
df_grouped[df_grouped['product_id'].isin([208079, 208127, 90344, 90470, 90475, 208082, 208089, 208086, 178951, 208087, 208081, 222549, 90292, 88999, 208083, 208138, 273051, 208080, 90492, 208084, 208090, 90111, 208085, 130073, 88777, 178641, 159658, 89231, 91120, 91545, 28273, 90136, 81399, 178826, 157453, 177096, 90209, 101999, 90370, 102260, 91016, 81268, 90317, 191289, 127180, 207286, 91098, 117617, 90174, 102127])].sort_values(by=['mrp'], ascending=False)

,product_id,mrp
23893,208127,11098880.0000
13421,90470,11049000.0000
23881,208079,9496300.0000
13307,88999,9037500.0000
13415,90344,8922000.0000
13311,89231,8757500.0000
12350,81399,7762800.0000
32470,273051,7360650.0000
5885,28273,6761450.0000
13422,90475,5109850.0000
